# 페이지 네비게이션

> 페이지 단위로 나누는 이유
- 효율적인 방법
- 검색한 데이터를 다 보내지 않고, 쪼개서 보내주는 개념


> DB에서 쉘을 통해 검색했을 때=> 2케어층(쉘과 db가 계속 연결되어있음.)
- 검색 결과가 10만건일 때
- 일단 몇개만 db가 보내주시고
- 페치: 나머지 데이터를 갖다주세요 요청하는 것.
- 데이터가 많을 때 불필요한 네트워크 리소스를 줄이는 방법
- cusor: 조그씩 데이터를 요청할 때마다 가지고 오는 것.
- 이미지 업로드할 때 chunk 단위로 받는 것과 같은 개념
- 64k, 100k등으로 쪼개서 주고 받음

> 3 케어 층?
- 구조: db- 클라이언트(django) - 웹서버 - 브라우저
- 접속이 끊어지기 때문에
- 데이터가 많을 때 브라우저로 모두 보낼 수 없기 때문에 
- 페이지 단위로 보내고, 사용자에게 네이게이션을 제공해주자
- 페이지를 이동할 수 있는 인터페이스가 필요.(네이게이션)
- 페이지 요청할 때마다 매번 select를 계속 실행함.
- SQL문을 작성할 때 페이지번호의 개념을 넣어줘야함.

# 1. 페이지 개념 익히기

In [ ]:
# urls.py
'''
from django.urls import path
from . import views 
from django.shortcuts import redirect

urlpatterns = [
    path('', views.page),
    path('<category>/<int:pk>/<mode>', views.BoardView.as_view(), name = 'myboard'),
    ]

'''

In [ ]:
# page.html
'''
{%for data in datas%}
{{data.id}} / {{data.name}} <br>

{% endfor %}
'''

## 1.1. 슬라이싱 개념으로 페이지 보여주기

In [ ]:
# myboard > views.py에서
'''
def page(request):
    datas = [{'id': 1, 'name':  '홍길동1'},
            {'id': 2, 'name':  '홍길동2'},
            {'id': 3, 'name':  '홍길동3'},
            {'id': 4, 'name':  '홍길동4'},
            {'id': 5, 'name':  '홍길동5'}]
    p = Paginator(datas, 3)# collections 형태의 데이터
                
                        # 한 페이지에 보여줄 데이터 개수
    sub = p.page(1) # 몇번째 페이지를 가지고 올 지 
    return render(request, 'myboard/page.html', {'datas': sub})
'''

## 1.2. 페이지 번호를 파라미터로 받아서 보여주기

In [ ]:
# view.py
'''
def page(request):
    datas = [{'id': 1, 'name':  '홍길동1'},
            {'id': 2, 'name':  '홍길동2'},
            {'id': 3, 'name':  '홍길동3'},
            {'id': 4, 'name':  '홍길동4'},
            {'id': 5, 'name':  '홍길동5'}]

    page = request.GET.get('page', 1)  #default: 1
    p = Paginator(datas, 3)# collections 형태의 데이터,  # 한 페이지에 보여줄 데이터 개수
    sub = p.page(page) # 몇번째 페이지를 가지고 올 지 
                    # 슬라이싱의 개념과 같음

    return render(request, 'myboard/page.html', {'datas': sub})
'''

In [ ]:
# page.html
'''
{%for data in datas%}
{{data.id}} / {{data.name}} <br>

{% endfor %}


<!-- 페이지 개수를 알고있는 경우 -->
<a href="?page = 1">처음으로</a>
<a href="?page = 1">[1]</a>
<a href="?page = 2">[2]</a>
<a href="?page =3">[3]</a>
<a href="?page =3">마지막으로</a>
'''

> 실행
- http://127.0.0.1:8000/myboard/?page=1

## 1.3. 현재 보고있는 page는 링크가 안걸려야 함.

> current page에 대한 정보
- 내 페이지는 링크가 안되기
- 이전으로/ 다음으로 링크 만들기
- 첫 페이지에서는 이전으로 개념: disable되어야 하고
- 마지막 페이지는 다음으로 개념이 disable되어야 함.
- Paginator에서 사용 가능

In [ ]:
# page.html
'''
{%for data in datas%}
{{data.id}} / {{data.name}} <br>

{% endfor %}
<br>
<br>



{% if datas.has_other_pages %}
    <!-- 이전 페이지 있는지 확인 -->

        {% if datas.has_previous %}
          <a href="?page={{ datas.previous_page_number }}">&laquo;</a>
        {% else %}
          <span>&laquo;</span>
        {% endif %}
<!-- 현재 보고 있는 페이지 확인
        index: 1부터 시작 -->

        {% for i in datas.paginator.page_range %}
          {% if datas.number == i %}
            <span>{{ i }} </span>
          {% else %}
            <a href="?page={{ i }}">{{ i }}</a>
          {% endif %}
        {% endfor %}
<!-- 다음 페이지가 있는지 확인 -->

        {% if datas.has_next %}
            <a href="?page={{ datas.next_page_number }}">&raquo;</a>
        {% else %}
            <span>&raquo;</span>
        {%endif %}
    {%endif%}



'''

In [ ]:
# # view.py
'''
def page(request):
    datas = [{'id': 1, 'name':  '홍길동1'},
            {'id': 2, 'name':  '홍길동2'},
            {'id': 3, 'name':  '홍길동3'},
            {'id': 4, 'name':  '홍길동4'},
            {'id': 5, 'name':  '홍길동5'},
            {'id': 6, 'name':  '홍길동6'},
            {'id': 7, 'name':  '홍길동7'},
            ]

    page = request.GET.get('page', 1)  #default: 1
    p = Paginator(datas, 3)# collections 형태의 데이터,  # 한 페이지에 보여줄 데이터 개수
    sub = p.page(page) # 몇번째 페이지를 가지고 올 지 
                    # 슬라이싱의 개념과 같음

    return render(request, 'myboard/page.html', {'datas': sub})
'''

# 2. BoardView()에 적용(게시판)

In [ ]:
# views.py
'''

elif mode == 'list':
            # pk값 상관없음.
            username = request.session.get('username', '')
            user = User.objects.get(username =username)
            data= models.Board.objects.all().filter(author = user, category = category)
            
            # page 개념
            page = request.GET.get('page', 1)  
            p = Paginator(data, 3)
            subs = p.page(page) 
            
            context = {'datas': subs,'username': username, 'category': category}
            return render (request, apps.APP + '/pagelist.html', context)

'''



In [ ]:
# pagelist.html
'''
<!-- import, includ 개념 -->
{% extends 'myboard/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

<!-- 게시물 출력 -->
    <br>
   
    <a href="{% url 'myboard'  category 0 'add' %}">Add </a> <br>
    여기부터 sub-template<br>

    {% for d in datas %}
    
            <a href="{% url 'myboard' category d.pk 'detail' %}"> {{d.title}} 조회수 {{d.cnt}}</a>
       <br>
    {% endfor %}


<!-- page 번호 출력 -->


{% if datas.has_other_pages %}
    <!-- 이전 페이지 있는지 확인 -->

        {% if datas.has_previous %}
          <a href="?page={{ datas.previous_page_number }}">&laquo;</a>
        {% else %}
          <span>&laquo;</span>
        {% endif %}
<!-- 현재 보고 있는 페이지 확인
        index: 1부터 시작 -->

        {% for i in datas.paginator.page_range %}
          {% if datas.number == i %}
            <span>{{ i }} </span>
          {% else %}
            <a href="?page={{ i }}">{{ i }}</a>
          {% endif %}
        {% endfor %}
<!-- 다음 페이지가 있는지 확인 -->

        {% if datas.has_next %}
            <a href="?page={{ datas.next_page_number }}">&raquo;</a>
        {% else %}
            <span>&raquo;</span>
        {%endif %}
    {%endif%}






{% endblock %}


'''